In [ ]:
!pip install fasttext

     |████████████████████████████████| 68 kB 3.4 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3145104 sha256=a53aec73eb72f9c66e2f2684b80d0a0703a70dfa4fcce6f36c850b6285586c23
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
import pandas as pd
import numpy as np
import os
import unicodedata
import torch
import json
import random
import fasttext
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score
import unicodedata

In [ ]:
YelpZip_file_path = '/content/drive/MyDrive/Course/CSE 573/CSE 573: Fake Reviews Project/YelpZip'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
metadata = pd.read_csv(os.path.join(YelpZip_file_path, 'metadata'), sep= '\t',
                      names=['user_id', 'prod_id','rating','label', 'date'], header=None)

with open(os.path.join(YelpZip_file_path, 'reviewContent')) as r:
  content = r.readlines()

### return: 
  list: dataset \\
  shape: (num_of_sample, 300) \\

  list: label  \\

  shaple (num_of_sample, 1) (with 0 and 1) \\

In [ ]:

number_of_samples = 50000

pretrained_model = fasttext.load_model('/content/drive/MyDrive/Course/CSE 573/CSE 573: Fake Reviews Project/cc.en.300.bin')



self_trained_model = fasttext.load_model('/content/drive/MyDrive/Course/CSE 573/CSE 573: Fake Reviews Project/trained_YelpZip_model.bin')

def fasttext_emb(content, metadata,ft_model):
  dataset = []
  label = []
  for text, lab in zip(content[:number_of_samples], metadata[:number_of_samples].itertuples(index=True, name='Pandas')):
    sentence = text.strip('\n').split('\t')[-1]

    sentence = unicodedata.normalize("NFKD", sentence)
    
    sentence_vector = ft_model.get_sentence_vector(sentence)
    dataset.append(sentence_vector)

    if lab.label == -1:
      l = 0
    else: 
      l = 1
    label.append(l)
  
  return dataset, label

In [ ]:
dataset,label = fasttext_emb(content, metadata,self_trained_model)
x_train, x_test, y_train, y_test = train_test_split(dataset, label, random_state=42,train_size=0.8) #train 80% test 20%


In [ ]:
len(x_train),len(x_test)

(40000, 10000)

In [ ]:
logit = LogisticRegression(max_iter=10000)
logit.fit(x_train, y_train)
y_pred = logit.predict(x_test)


In [ ]:
print("accuracy_score: ", accuracy_score(y_test,y_pred))
print("precision_score: ", precision_score(y_test,y_pred, pos_label=1))
print("recall_score: ", recall_score(y_test,y_pred, pos_label=1))
print("f1_score: ", f1_score(y_test,y_pred, pos_label=1))

accuracy_score:  0.8634
precision_score:  0.8652860158841862
recall_score:  0.996988300706591
f1_score:  0.9264800861141012
